In [1]:
run ../data_utils.py

In [49]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords

In [35]:
def tables_from_category(category):
    with open('../../data/pickle_files/%s.pickle' % category, 'rb') as f:
        return pickle.load(f)

qa_table, reviews_table = tables_from_category(ELECTRONICS)

#Question with maximum length

In [38]:
mask = qa_table['questionText'].str.len() == 19755
result = qa_table.loc[mask]['questionText']
question = result.iloc[1]

In [43]:
stopset_u = set(stopwords.words('english'))
stopset = set([str(sw) for sw in stopset_u])

tokens = nltk.word_tokenize(question)
tokens = [w.lower() for w in tokens if not w.lower() in stopset]

print len(tokens)

#Questions containing "Read More"

In [53]:
mask = qa_table['questionText'].str.contains("Read More")
result = qa_table.loc[mask]['questionText']

In [58]:
len(result)

29291